In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import re
import json
import requests

import yaml
import random
import pandas as pd
from tqdm.auto import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

Mounted at /content/drive


### Collect a List of Possible Python Notebook Candidates

In [ ]:
def read_json(filepath):
  return json.load(open(filepath))

def read_ipynb(file): 
  with open(file, mode= 'r', encoding= 'utf-8') as f:
    return json.loads(f.read())

random_seed = 30
!wget https://github-notebooks-samples.s3-eu-west-1.amazonaws.com/ntbs_list.json
notebooks_list = read_json('ntbs_list.json')
random.Random(random_seed).shuffle(notebooks_list)

--2022-07-24 21:03:53--  https://github-notebooks-samples.s3-eu-west-1.amazonaws.com/ntbs_list.json
Resolving github-notebooks-samples.s3-eu-west-1.amazonaws.com (github-notebooks-samples.s3-eu-west-1.amazonaws.com)... 52.218.30.216
Connecting to github-notebooks-samples.s3-eu-west-1.amazonaws.com (github-notebooks-samples.s3-eu-west-1.amazonaws.com)|52.218.30.216|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 536816054 (512M) [application/json]
Saving to: ‘ntbs_list.json’

ntbs_list.json      100%[===================>] 511.95M  29.4MB/s    in 19s     

2022-07-24 21:04:13 (27.2 MB/s) - ‘ntbs_list.json’ saved [536816054/536816054]



In [1]:
def is_python_notebook(nb_file):
  try:
    nb_ver = nb_file['metadata']['language_info']['version']
    nb_lang = nb_file['metadata']['kernelspec']['name']
    version_3 = nb_ver.startswith('3.') 
    in_python = bool(re.search('python', nb_lang))
    return version_3 and in_python 
  except:
    return False

def has_min_cell_types(nb_file):
  md_count = 0
  code_count = 0
  try:
    for cell in nb_file['cells']:
      if cell['source'] == []:
        pass
      elif cell['cell_type'] == 'markdown':
        md_count += 1
      elif cell['cell_type'] == 'code':
        code_count += 1
  except:
    return False
  return (code_count >= 1 and md_count >= 1)

def is_ml_related(nb_file):
  ml_pattern = re.compile(r'sklearn|keras|tensorflow|pytorch|torch|seaborn|matplotlib|numpy|pandas|scipy|xgboost|lightgbm|fastai|kaggle')
  for cell in nb_file['cells']:
    try:
      if bool(re.search(ml_pattern, cell['source'][0])):
        return True
    except:
      pass
  return False

def useable_notebook(nb_file):
  if is_python_notebook(nb_file):
    if has_min_cell_types(nb_file):
      if is_ml_related(nb_file):
        return True
  return False

In [ ]:
# Filter notebooks list

# Around 25-30% of the notebooks are worthy
# To generate 400,000 notebooks you need to go through around 1.21 million files

python_nbs = []

with requests.Session() as s:
  for nb in tqdm(notebooks_list[429207:1210000], desc='Collecting Python Notebooks'):
    response = s.get(f'https://github-notebooks-update1.s3-eu-west-1.amazonaws.com/{nb}')
    try:
      response.raise_for_status()
      nb_file = response.json()
    except:
      continue
    if useable_notebook(nb_file):
      current_doc = pd.DataFrame(nb_file['cells'])

In [ ]:
extra_data_path = '/content/drive/MyDrive/Colab Notebooks/ML Projects/Google AI4Code/data/jup_python_ids/extra_ids_429207_500586.yaml'

python_nbs = list(set(python_nbs))
with open(extra_data_path, 'w') as outfile:
  yaml.dump(python_nbs, outfile, default_flow_style=False)

print(f"Successfully created yaml file for {extra_data_path}.")

Successfully created yaml file for /content/drive/MyDrive/Colab Notebooks/ML Projects/Google AI4Code/data/jup_python_ids/extra_ids_429207_500586.yaml.
